In [ ]:
import copy

from board import Board
from io_helper import Menu, Option
from utils import query_player
from minimax import minimax_search
from mcts import monte_carlo_tree_search

In [2]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError
        

def play_game(game, strategies: dict, verbose=True):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        if verbose:
            print(f"Player {player} move: {move}")  # Debugging log
        try:
            state = game.result(state, move)
        except Exception as e:
            print(f"Error processing move {move}: {e}")  # Debugging log
            raise
    
    current_player = state.to_move
    result = game.utility(state, current_player)
    if result > 0:
        print(f"Player {current_player} wins!")
    elif result < 0:
        print(f"Player {current_player} loses.")
    else:
        print("It's a draw.")

In [3]:
def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)

In [4]:
class Checkers(Game):
    def __init__(self, players):
        """
        players: a list of two identifiers (e.g., ["White", "Black"])
        The state is a dict with keys:
          - 'board': an instance of Board
          - 'turn': whose turn it is to move
        """
        self.players = players
        self.initial = self.create_initial_state()
    
    def create_initial_state(self):
        board = Board(players=self.players, to_move=self.players[0])
        board.setup()
        # Set initial turn to players[0]
        return board
    
    def actions(self, state):
        """
        Return a list of valid moves for the current state's turn.
        Each move is represented as a tuple: ((from_row, from_col), (to_row, to_col)).
        For simplicity, multi-jump moves are not implemented.
        """
        valid_moves = []
        board = state.grid
        current = state.to_move
        forced_piece = state.forced_piece

        if forced_piece is None:
            for row in range(8):
                for col in range(8):
                    piece = board[row][col]
                    if piece is not None and piece.player == current:
                        valid_moves.extend(self.get_valid_moves(board, piece))
        else:
            jumps = self.get_jumps(board, forced_piece)
            if len(jumps) > 0:
                valid_moves.extend(jumps)
        return valid_moves
    
    def get_valid_moves(self, board, piece):
        """
        Return valid destination positions for the piece at pos.
        It uses the piece's available_moves method and then checks the board.
        If the destination square is occupied by an opponent, a jump move is considered.
        """
        moves = []
        # Get candidate moves from the piece.
        for candidate in piece.available_moves():
            _, new_row, new_col = candidate
            if board[new_row][new_col] is None:
                moves.append(candidate)
            else:
                jump_candidate = self.get_valid_jump(board, candidate, piece)
                if jump_candidate is not None:
                    moves.append(jump_candidate)
        return moves
    
    def get_valid_jump(self, board, candidate, piece):
        piece_id, row, col = candidate
        
        # If the adjacent square has an opponent piece, try a jump move.
        if board[row][col] is not None and board[row][col].player != piece.player:
                    jump_candidate = piece.jump_move(candidate)
                    if jump_candidate is not None:
                        jump_row, jump_col = jump_candidate[1], jump_candidate[2]
                        if board[jump_row][jump_col] is None:
                            return (piece_id, jump_row, jump_col)
        return None

    
    def result(self, state, move):
        """
        Given a state and a move ((from_row, from_col), (to_row, to_col)),
        return a new state with the move executed.
        This includes moving the piece, capturing any jumped piece,
        promoting the piece (if applicable), and switching the turn.
        """
        new_state = copy.deepcopy(state)
        board_obj = new_state
        board = board_obj.grid
        piece_id, to_row, to_col = move
        piece = board_obj.get_piece_by_id(piece_id)
        from_row, from_col = piece.cy, piece.cx
        
        # Move the piece using its own move method.
        piece.move((to_row, to_col))
        board[to_row][to_col] = piece
        board[from_row][from_col] = None

        # Handle capturing: if the move spans two rows and columns, remove the jumped piece.
        if abs(from_row - to_row) == 2 and abs(from_col - to_col) == 2:
            mid_row = (from_row + to_row) // 2
            mid_col = (from_col + to_col) // 2
            board[mid_row][mid_col] = None
            jumps = self.get_jumps(board, piece)
            if len(jumps) > 0:
                new_state.forced_piece = piece
                return new_state

        new_state.forced_piece = None
        new_state.to_move = self.players[1] if state.to_move == self.players[0] else self.players[0]
        return new_state

    def is_terminal(self, state):
        """
        The game is terminal if one player has no pieces left or if the current
        player has no valid moves.
        """
        board = state.grid
        pieces_count = {self.players[0]: 0, self.players[1]: 0}
        for row in board:
            for cell in row:
                if cell is not None:
                    pieces_count[cell.player] += 1
        if pieces_count[self.players[0]] == 0 or pieces_count[self.players[1]] == 0:
            return True
        
        # Terminal if current player has no moves.
        if not self.actions(state):
            return True
        
        return False
    
    def utility(self, state, player):
        """
        Return 1 if the given player wins, -1 if the player loses,
        and 0 for a tie or non-terminal state.
        This assumes that the state is terminal.
        """
        if not self.is_terminal(state):
            return 0
        board = state.grid
        pieces_count = {self.players[0]: 0, self.players[1]: 0}
        for row in board:
            for cell in row:
                if cell is not None:
                    pieces_count[cell.player] += 1
        
        if pieces_count[self.players[0]] == 0:
            return 1 if player == self.players[1] else -1
        if pieces_count[self.players[1]] == 0:
            return 1 if player == self.players[0] else -1
        
        # If the game ended due to no moves, consider it a loss for the player who cannot move.
        if len(self.actions(state)) == 0:
            return -1
        
        return 0
    
    def get_jumps(self, board, piece):
        valid_moves = []
        for candidate in piece.available_moves():
                jump_candidate = self.get_valid_jump(board, candidate, piece)
                if jump_candidate is not None:
                    valid_moves.append(jump_candidate)
        return valid_moves

    def display(self, state):
        """
        Display the board in a user-friendly format.
        """
        state.print_board()

In [5]:
def start_checkers_game():
    print("Welcome to the Checkers Game!")
    menu = Menu()
    menu.display_menu()
    user_choice = menu.read_user_input()

    minimax_player = player(minimax_search)

    strategies = {
        "w": query_player,
        "b": player(monte_carlo_tree_search)
    }

    if user_choice == Option.HUMAN_VS_MINIMAX_AI.value:
        strategies["b"] = minimax_player
    elif user_choice == Option.HUMAN_VS_HUMAN.value:
        strategies["b"] = query_player
    elif user_choice == Option.MINIMAX_AI_VS_MCTS_AI.value:
        strategies["w"] = minimax_player

    play_game(Checkers(["w", "b"]), strategies)

In [6]:
start_checkers_game()

Welcome to the Checkers Game!
1. Human vs Minimax AI
2. Human vs MCTS AI
3. Human vs Human
4. Minimax AI vs MCTS AI
0. Quit


Choice (0-4):  1


current state:
     +-----+-----+-----+-----+-----+-----+-----+-----+
1    |     | b01 |     | b02 |     | b03 |     | b04 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
2    | b05 |     | b06 |     | b07 |     | b08 |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
3    |     | b09 |     | b10 |     | b11 |     | b12 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
4    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
5    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
6    | w01 |     | w02 |     | w03 |     | w04 |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
7    |     | w05 |     | w06 |     | w07 |     | w08 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
8    | w09 |     | w10 |     | w11 |     | w12 |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+

        A     B     C     D     E     F     G    

Choice (0-7):  1


Player w move: ('w01', 4, 1)
Player b move: ('b09', 3, 0)
current state:
     +-----+-----+-----+-----+-----+-----+-----+-----+
1    |     | b01 |     | b02 |     | b03 |     | b04 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
2    | b05 |     | b06 |     | b07 |     | b08 |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
3    |     |     |     | b10 |     | b11 |     | b12 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
4    | b09 |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
5    |     | w01 |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
6    |     |     | w02 |     | w03 |     | w04 |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
7    |     | w05 |     | w06 |     | w07 |     | w08 |
     +-----+-----+-----+-----+-----+-----+-----+-----+
8    | w09 |     | w10 |     | w11 |     | w12 |     |
     +-----+-----+-----+-----+-----+-----+-----

...

Player w move: ('w12', 5, 4)
Player b move: ('b05', 6, 1)
current state:
     +-----+-----+-----+-----+-----+-----+-----+-----+
1    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
2    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
3    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
4    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
5    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
6    |     |     | W04 |     | W12 |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
7    |     | B05 |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
8    |     |     | B09 |     | W08 |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----

Choice (0-10):  1


Player w move: ('w04', 7, 0)
Player b move: ('b09', 6, 1)
current state:
     +-----+-----+-----+-----+-----+-----+-----+-----+
1    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
2    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
3    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
4    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
5    |     |     |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
6    |     |     |     |     | W12 |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
7    |     | B09 |     |     |     |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----+-----+
8    | W04 |     |     |     | W08 |     |     |     |
     +-----+-----+-----+-----+-----+-----+-----

Choice (0-7):  5


Player w move: ('w04', 5, 2)
Player b loses.
